In [1]:
# !pip install --upgrade pip
# !pip install pandas
# !cp ./fp_refined.csv /data

In [2]:
%matplotlib inline

import numpy as np
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt


import pandas as pd

from tqdm.notebook import tnrange, tqdm

# tf.enable_eager_execution()  # tf2

In [3]:
def read_mono_from_image_unicode(path):
    """workaround for non-ascii filenames"""

    stream = open(path, "rb")
    bytes = bytearray(stream.read())
    numpyarray = np.asarray(bytes, dtype=np.uint8)
    mono = cv2.imdecode(numpyarray, cv2.IMREAD_UNCHANGED)

    return mono


def fp_float_from_mono(mono):
    """create 0 or 1 binary mask image from 
    wall/entrance/LDK/bedroom/balcony/bathroom stacked array"""

    # AREA_WALL = 64
    # AREA_ENTRANCE = 32
    # AREA_LDK = 16
    # AREA_BEDROOM = 8
    # AREA_BALCONY = 4
    # AREA_BATHROOM = 2

    mask_bits = np.array([64, 32, 16, 8, 4, 2], dtype=np.uint8)
    mask = np.broadcast_to(mask_bits, (*mono.shape[:2], 6))

    unit_comb = (((np.expand_dims(mono, 2) & mask) > 0)).astype(np.float)

    return unit_comb


def pad_fp(fp, width=112, height=112):
    """place the fp at the bottom center of padded image."""
    h, w = np.subtract(fp.shape[:2], (height, width))
    if h > 0:
        fp = fp[h : h + height, :, :]
    if w > 0:
        fp = fp[:, w // 2 : w // 2 + width, :]

    h, w = np.subtract((height, width), fp.shape[:2])
    fp = np.pad(fp, ((max(h, 0), 0), (max(w // 2, 0), max(w - w // 2, 0)), (0, 0)))
    return fp


def visualize_fp(fps):
    # adjusted for different luminance
    channel_to_rgba = np.array(
        [
            [0.0, 0.0, 0.0, 0.0],  # wall to black L0
            [0.0, 0.33, 0.0, 0.0],  # entrance to green L30
            [1.0, 0.25, 0.0, 0.0],  # LDK to red L57
            [0.83, 0.87, 0.0, 0.0],  # bedroom to yellow L85
            [0.0, 0.26, 1.0, 0.0],  # balcony to blue L40
            [0.0, 0.81, 0.76, 0.0],  # bathroom to cyan L75
        ]
    )

    # make colors subtractive
    channel_to_rgba[:, 0:3] -= 1

    # put it on white
    fps_rgba = np.clip(
        np.array([1.0, 1.0, 1.0, 1.0]) + (np.array(fps) @ channel_to_rgba), 0, 1
    )
    return fps_rgba


# def _fp_from_string(bytes):
#     return np.frombuffer(bytes).reshape(28,28,6)

In [21]:
from os import makedirs


# 10001_57B
# 2112_49_0


def visualize_from_file(ids, dir_to="vis"):

    makedirs(dir_to, exist_ok=True)

    for id in ids:
        mono = read_mono_from_image_unicode(f"/data/fp_img_processed/{id}.png")
        fp = fp_float_from_mono(mono)
        fp_rgba = visualize_fp(fp)
        
        fp_light = cv2.cvtColor(fp_rgba.astype(np.uint8)*255, cv2.COLOR_RGB2Lab)[:, :, 0] / 100
        fp_rgba = pad_fp(fp_rgba, fp_light.shape[1], fp_light.shape[0])

        fig = plt.figure(figsize=(5, 5), dpi=300)
        plt.imshow(fp_rgba)
        plt.axis("off")

        plt.tight_layout()
        fig.savefig(f"{dir_to}/{id}.png", bbox_inches="tight", pad_inches=0)
        plt.close()

visualize_from_file(['10001_57B'])

In [22]:
from pathlib import Path
import csv

def read_from_csv(filepath, columns=False):
    if Path(filepath).is_file():

        with open(filepath, "r", newline="", encoding="utf-8-sig") as csvfile:
            listreader = csv.reader(csvfile)
            if columns:
                columns = next(listreader)
            readlist = list(listreader)

    else:
        columns = []
        readlist = []

    if columns:
        return columns, readlist
    else:
        return readlist
    
top_ids = read_from_csv("vgg_activation_top10.csv")
top_ids

[[],
 [],
 [],
 ['4976_62',
  '5039_105',
  '199_86',
  '317_96',
  '10667_66',
  '5504_70',
  '5814_94',
  '2112_49_0',
  '959_89A',
  '2430_58'],
 ['7963_180B',
  '100754_163B',
  '2888_118A',
  '2893_118A',
  '7963_118A',
  '6338_162',
  '12685_172A',
  '2923_121',
  '108_168',
  '15045_109B'],
 ['106355_102',
  '2102_82',
  '1183_199',
  '2672_162',
  '16429_107',
  '8208_77',
  '2836_121',
  '2401_101',
  '3896_103',
  '6519_105'],
 ['10415_106A2',
  '23950_102',
  '23956_102',
  '23961_103',
  '8289_182',
  '10422_86',
  '15156_129',
  '9106_154',
  '3376_107B',
  '25022_77B'],
 ['25507_151',
  '104253_110D1',
  '104253_110D',
  '26013_109',
  '100995_109B',
  '25765_83B',
  '105202_119A',
  '13160_110C',
  '19189_198A',
  '26216_111C'],
 ['105024_113A',
  '105322_112C',
  '104895_109A',
  '104893_137B',
  '103531_82B',
  '105377_123',
  '103537_114B1',
  '104986_111F',
  '105812_128B',
  '103547_108A'],
 ['109468_111A1',
  '107330_82C',
  '115505_123A',
  '109215_123C',
  '10814

In [23]:
ids_flat = [id for line in top_ids for id in line]
ids_flat

['4976_62',
 '5039_105',
 '199_86',
 '317_96',
 '10667_66',
 '5504_70',
 '5814_94',
 '2112_49_0',
 '959_89A',
 '2430_58',
 '7963_180B',
 '100754_163B',
 '2888_118A',
 '2893_118A',
 '7963_118A',
 '6338_162',
 '12685_172A',
 '2923_121',
 '108_168',
 '15045_109B',
 '106355_102',
 '2102_82',
 '1183_199',
 '2672_162',
 '16429_107',
 '8208_77',
 '2836_121',
 '2401_101',
 '3896_103',
 '6519_105',
 '10415_106A2',
 '23950_102',
 '23956_102',
 '23961_103',
 '8289_182',
 '10422_86',
 '15156_129',
 '9106_154',
 '3376_107B',
 '25022_77B',
 '25507_151',
 '104253_110D1',
 '104253_110D',
 '26013_109',
 '100995_109B',
 '25765_83B',
 '105202_119A',
 '13160_110C',
 '19189_198A',
 '26216_111C',
 '105024_113A',
 '105322_112C',
 '104895_109A',
 '104893_137B',
 '103531_82B',
 '105377_123',
 '103537_114B1',
 '104986_111F',
 '105812_128B',
 '103547_108A',
 '109468_111A1',
 '107330_82C',
 '115505_123A',
 '109215_123C',
 '108144_99C',
 '109497_87C',
 '108805_94B',
 '107429_130A',
 '109395_94B',
 '107324_109A']

In [24]:
visualize_from_file(ids_flat)